In [5]:
# where is it ?

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import TargetEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from catboost import CatBoostClassifier

In [6]:
train_df_full = pd.read_csv('train.csv')#'/kaggle/input/playground-series-s4e9/train.csv')
test_df_full = pd.read_csv('test.csv')#'/kaggle/input/playground-series-s4e9/test.csv')
SEED = 3

In [7]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

def process_categoricals_cv(X_train, X_test, y_train, method, target_col=None, smoothing=0.3):
    """Encode categorical variables based on the chosen method."""
    
    if method == 'integer':
        # Simple Integer Encoding
        encoder = LabelEncoder()
        X_train_encoded = X_train.copy()
        X_test_encoded = X_test.copy()
        for col in X_train.select_dtypes(include=['object', 'category']).columns:
            encoder.fit(X_train[col])
            X_train_encoded[col] = encoder.transform(X_train[col])
            X_test_encoded[col] = encoder.transform(X_test[col])
    
    elif method == 'binary':
        # Binary Encoding
        encoder = ce.BinaryEncoder(cols=X_train.select_dtypes(include=['object', 'category']).columns)
        X_train_encoded = encoder.fit_transform(X_train)
        X_test_encoded = encoder.transform(X_test)
    
    elif method == 'kfold_target':
        # K-Fold Target Encoding to avoid data leakage
        encoder = ce.TargetEncoder(cols=X_train.select_dtypes(include=['object', 'category']).columns)
        X_train_encoded = X_train.copy()
        X_test_encoded = X_test.copy()
        kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
        
        for train_idx, valid_idx in kf.split(X_train):
            X_train_fold, X_valid_fold = X_train.iloc[train_idx], X_train.iloc[valid_idx]
            y_train_fold = y_train.iloc[train_idx]

            # Fit encoder on the fold and transform
            encoder.fit(X_train_fold, y_train_fold)
            X_train_encoded.iloc[valid_idx] = encoder.transform(X_valid_fold)
        
        # Fit encoder on the entire training data and apply it to test data
        encoder.fit(X_train, y_train)
        X_test_encoded = encoder.transform(X_test)
    
    elif method == 'smoothed_target':
        # Smoothed Target Encoding with Smoothing Factor
        encoder = ce.TargetEncoder(cols=X_train.select_dtypes(include=['object', 'category']).columns, smoothing=smoothing)
        X_train_encoded = encoder.fit_transform(X_train, y_train)
        X_test_encoded = encoder.transform(X_test)
    
    else:
        raise ValueError("Invalid encoding method. Choose from 'integer', 'binary', 'kfold_target', or 'smoothed_target'.")
    
    return X_train_encoded, X_test_encoded


In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def cross_validate_model(X, y, encoding_method, model, n_splits=5, smoothing=0.3):
    """Perform cross-validation with specified encoding method and model."""
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    scores = []
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Apply encoding to categorical variables
        X_train_encoded, X_test_encoded = process_categoricals_cv(X_train, X_test, y_train, method=encoding_method, smoothing=smoothing)
        
        # Train the model
        model.fit(X_train_encoded, y_train)
        
        # Evaluate the model using ROC AUC score
        y_pred_prob = model.predict_proba(X_test_encoded)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_prob)
        scores.append(roc_auc)
    
    return np.mean(scores), np.std(scores)



# comparing different encoding methods

In [19]:
# Example usage of the pipeline
TARGET = 'loan_status'

# Prepare the data
dev_frac = .5
df_dev = train_df_full.sample(frac=dev_frac, random_state=SEED)
X = df_dev.drop(columns=TARGET)
y = df_dev[TARGET]

# Initialize model
model = CatBoostClassifier(verbose=0, random_state=SEED)

# Define the encoding methods
encoding_methods = ['integer', 'binary', 'kfold_target', 'smoothed_target']

# Store results
cv_results = {}

# Perform cross-validation for each encoding method
for method in encoding_methods:
    print(f"Running cross-validation with {method} encoding...")
    mean_score, std_score = cross_validate_model(X, y, encoding_method=method, model=model, n_splits=5)
    cv_results[method] = {'mean_score': mean_score, 'std_score': std_score}
    print(f"{method} encoding: Mean ROC AUC = {mean_score:.4f}, Std = {std_score:.4f}")

# Output the results
print("\nCross-validation results for different encoding methods:")
for method, result in cv_results.items():
    print(f"{method}: Mean ROC AUC = {result['mean_score']:.4f}, Std = {result['std_score']:.4f}")


Running cross-validation with integer encoding...
integer encoding: Mean ROC AUC = 0.9486, Std = 0.0061
Running cross-validation with binary encoding...
binary encoding: Mean ROC AUC = 0.9471, Std = 0.0067
Running cross-validation with kfold_target encoding...
kfold_target encoding: Mean ROC AUC = 0.9435, Std = 0.0083
Running cross-validation with smoothed_target encoding...
smoothed_target encoding: Mean ROC AUC = 0.9482, Std = 0.0074

Cross-validation results for different encoding methods:
integer: Mean ROC AUC = 0.9486, Std = 0.0061
binary: Mean ROC AUC = 0.9471, Std = 0.0067
kfold_target: Mean ROC AUC = 0.9435, Std = 0.0083
smoothed_target: Mean ROC AUC = 0.9482, Std = 0.0074


# comparing different smoothing params 

In [16]:
# Example usage of the pipeline
TARGET = 'loan_status'

# Prepare the data
dev_frac = .4
df_dev = train_df_full.sample(frac=dev_frac, random_state=SEED)
X = df_dev.drop(columns=TARGET)
y = df_dev[TARGET]

# Initialize model
model = CatBoostClassifier(verbose=0, random_state=SEED)

# Define the encoding methods
smoothing_numbers = [.1, .2, .3, .4, .5]

# Store results
cv_results = {}

# Perform cross-validation for each encoding method
for smoothing in smoothing_numbers:
    print(f"Running cross-validation with {smoothing} encoding...")
    mean_score, std_score = cross_validate_model(X, y, encoding_method='smoothed_target', model=model, n_splits=5, smoothing=smoothing)
    cv_results[smoothing] = {'mean_score': mean_score, 'std_score': std_score}
    print(f"{smoothing} encoding: Mean ROC AUC = {mean_score:.4f}, Std = {std_score:.4f}")

# Output the results
print("\nCross-validation results for different encoding methods:")
for method, result in cv_results.items():
    print(f"{method}: Mean ROC AUC = {result['mean_score']:.4f}, Std = {result['std_score']:.4f}")


Running cross-validation with 0.1 encoding...
0.1 encoding: Mean ROC AUC = 0.9487, Std = 0.0040
Running cross-validation with 0.2 encoding...
0.2 encoding: Mean ROC AUC = 0.9487, Std = 0.0040
Running cross-validation with 0.3 encoding...
0.3 encoding: Mean ROC AUC = 0.9487, Std = 0.0040
Running cross-validation with 0.4 encoding...
0.4 encoding: Mean ROC AUC = 0.9487, Std = 0.0040
Running cross-validation with 0.5 encoding...
0.5 encoding: Mean ROC AUC = 0.9487, Std = 0.0040

Cross-validation results for different encoding methods:
0.1: Mean ROC AUC = 0.9487, Std = 0.0040
0.2: Mean ROC AUC = 0.9487, Std = 0.0040
0.3: Mean ROC AUC = 0.9487, Std = 0.0040
0.4: Mean ROC AUC = 0.9487, Std = 0.0040
0.5: Mean ROC AUC = 0.9487, Std = 0.0040


# producing the output

In [9]:
import category_encoders as ce
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder


def run_submission(train_df_full, test_df_full, encoding_method, model, target_col='loan_status', submission_file_name=None):
    """Run the submission pipeline for a given encoding method."""
    
    # Step 1: Separate features (X) and target (y) from the full training set
    X_train = train_df_full.drop(columns=[target_col, 'id'])
    y_train = train_df_full[target_col]
    X_test = test_df_full.drop(columns='id')
    
    # Step 3: Process categorical variables based on the chosen method
    X_train_encoded, X_test_encoded = process_categoricals_cv(X_train, X_test, y_train, method=encoding_method)

    # Step 4: Create and fit the model on the encoded training data
    model.fit(X_train_encoded, y_train)

    # Step 6: Make predictions
    predictions = model.predict_proba(X_test_encoded)[:, 1]  # Probabilities for class 1

    # Step 7: Create a submission DataFrame
    submission = pd.DataFrame({
        'id': test_df_full['id'],  # Preserve 'id' from the test set
        target_col: predictions  # The predicted probabilities (logits percentage)
    })

    # Step 8: Save the submission to a CSV file
    submission_file = submission_file_name or f'submissions/submission2ndRound_{encoding_method}_DefaultCatboost.csv'
    submission.to_csv(submission_file, index=False)
    print(f"Submission saved as {submission_file}")


# Example usage for each encoding method

# Initialize CatBoost model
model = CatBoostClassifier(verbose=0, random_state=SEED)

# Define the encoding methods
encoding_methods = ['integer', 'binary', 'kfold_target', 'smoothed_target']

# Loop over each encoding method and create a submission
for method in encoding_methods:
    run_submission(train_df_full, test_df_full, encoding_method=method, model=model)


Submission saved as submissions/submission2ndRound_integer_DefaultCatboost.csv
Submission saved as submissions/submission2ndRound_binary_DefaultCatboost.csv
Submission saved as submissions/submission2ndRound_kfold_target_DefaultCatboost.csv
Submission saved as submissions/submission2ndRound_smoothed_target_DefaultCatboost.csv
